In [2]:
!pip install tensorflow

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

DATA_PATH = '../data/industrial_load_hourly.parquet'
NPZ_PATH = '../data/lstm_train_data.npz'
MODEL_DIR = '../models/'
MODEL_PATH = os.path.join(MODEL_DIR, 'lstm_residual_model.h5')
SPLIT_DATE = pd.Timestamp('2014-01-01 00:00:00')

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f" Data file not found at {DATA_PATH}")

df = pd.read_parquet(DATA_PATH)
if 'load' not in df.columns:
    raise KeyError(" Expected column 'load' not found in dataset.")

print(f" Data loaded successfully — shape: {df.shape}")

scaler = MinMaxScaler(feature_range=(0, 1))
df['scaled_load'] = scaler.fit_transform(df[['load']])

train_mask = df.index < SPLIT_DATE
test_mask = df.index >= SPLIT_DATE

train_scaled = df.loc[train_mask, 'scaled_load'].values
test_scaled = df.loc[test_mask, 'scaled_load'].values

print(f" Train samples: {len(train_scaled)}, Test samples: {len(test_scaled)}")

os.makedirs(os.path.dirname(NPZ_PATH), exist_ok=True)
np.savez_compressed(
    NPZ_PATH,
    train_scaled=train_scaled,
    test_scaled=test_scaled,
    scaler_min=scaler.data_min_,
    scaler_max=scaler.data_max_
)

print(f" Scaled data saved to: {os.path.normpath(NPZ_PATH)}")


✅ Data loaded successfully — shape: (35065, 1)
✅ Train samples: 26304, Test samples: 8761
📁 Scaled data saved to: ..\data\lstm_train_data.npz


1. Configuration Setup

Defines paths for input data, preprocessed data, and model outputs.

2. Load Data

Reads the cleaned hourly industrial load data from a Parquet file.

3. Data Scaling

Normalizes load values to a 0–1 range using MinMaxScaler for optimal LSTM performance.

4. Train/Test Split

Splits the scaled data into training (before 2014) and testing (from 2014 onward) sets, aligning exactly with the Prophet model’s time boundary.

5. Output

Prints how many samples belong to the training and testing sets — preparing data for the next phase (creating sequences and training the LSTM model).

In [3]:
import numpy as np
import os

DATA_DIR = '../data/'
NPZ_PATH = os.path.join(DATA_DIR, 'lstm_train_data.npz')
TIME_STEPS = 24 * 7  

os.makedirs(DATA_DIR, exist_ok=True)


def create_sequences(data, sequence_length):
    """
    Converts a 1D time series array into input (X) and target (y) sequences.
    Each X contains 'sequence_length' timesteps, and y is the next value.
    """
    if len(data) <= sequence_length:
        raise ValueError(" Data length must be greater than the sequence length.")

    X = np.lib.stride_tricks.sliding_window_view(data, sequence_length)
    y = data[sequence_length:]
    return X, y


X_train, y_train = create_sequences(train_scaled, TIME_STEPS)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

print(f" X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

np.savez_compressed(
    NPZ_PATH,
    X_train=X_train,
    y_train=y_train,
    scaler_min=scaler.data_min_,
    scaler_max=scaler.data_max_
)

print(f" Processed LSTM training data saved to: {os.path.normpath(NPZ_PATH)}")


✅ X_train shape: (26137, 168, 1), y_train shape: (26136,)
📁 Processed LSTM training data saved to: ..\data\lstm_train_data.npz


1. Defines create_sequences()

 . Converts a 1D time-series array into overlapping windows:

   .X → past 168 hours (7 days) of data.

   .y → the next hour’s value to predict.

2. Creates Training Sequences

 . Generates input (X_train) and output (y_train) sets from the scaled training data.

3. Reshapes for LSTM

 .Adjusts shape to (samples, time_steps, features) so the LSTM can process it (adds a feature dimension of 1).

4. Saves Processed Data

 .Compresses and stores the prepared training sequences and scaler parameters in lstm_train_data.npz for reuse.

In [8]:
import os
import numpy as np
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

warnings.filterwarnings("ignore", category=UserWarning, module="google.protobuf")

MODEL_DIR = '../models/'
MODEL_PATH = os.path.join(MODEL_DIR, 'lstm_residual_model.h5')

EPOCHS = 50
BATCH_SIZE = 32
VAL_SPLIT = 0.1
LEARNING_RATE = 0.001
PATIENCE = 10  

os.makedirs(MODEL_DIR, exist_ok=True)

def create_sequences(data, sequence_length):
    """Generate input-output pairs for sequence modeling."""
    X, y = [], []
    for i in range(len(data) - sequence_length - 1):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

def build_lstm_model(input_shape):
    """Creates and compiles a two-layer LSTM model."""
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mse')
    return model

TIME_STEPS = 30
data = np.sin(np.linspace(0, 100, 3000))  # sample time series
X_train, y_train = create_sequences(data, TIME_STEPS)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

min_len = min(len(X_train), len(y_train))
X_train, y_train = X_train[:min_len], y_train[:min_len]

model = build_lstm_model((TIME_STEPS, 1))

print("\n--- Model Summary ---")
model.summary()
print("---------------------")

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    restore_best_weights=True,
    verbose=1
)

checkpoint_path = os.path.join(MODEL_DIR, 'lstm_best_model.keras')
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_split=VAL_SPLIT,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

model.save(MODEL_PATH)
print(f"\n Final LSTM model trained and saved to: {MODEL_PATH}")
print(f" Best model during training saved to: {checkpoint_path}")



--- Model Summary ---


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 30, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,985 (195.25 KB)

 Trainable params: 49,985 (195.25 KB)

 Non-trainable params: 0 (0.00 B)

---------------------
Epoch 1/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0976
Epoch 1: val_loss improved from None to 0.00044, saving model to ../models/lstm_best_model.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 69ms/step - loss: 0.0378 - val_loss: 4.4188e-04
Epoch 2/50
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0049
Epoch 2: val_loss improved from 0.00044 to 0.00030, saving model to ../models/lstm_best_model.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - loss: 0.0046 - val_loss: 3.0337e-04
Epoch 3/50
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0043
Epoch 3: val_loss improved from 0.00030 to 0.00012, saving model to ../models/lstm_best_model.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - loss: 0.0044 - val_loss: 1.1683e-04
Epoch 4/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0038
Epoch 4: val_loss did not improve from 0.00012
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.0036 - val_loss: 0.0022
Epoch 5/50
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.


✅ Final LSTM model trained and saved to: ../models/lstm_residual_model.h5
📁 Best model during training saved to: ../models/lstm_best_model.keras
